# 보팅 (xgboost + lgbm + rf)

In [1]:
import pandas as pd

import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
train

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X,train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# pycaret

In [4]:
from pycaret.classification import *

In [5]:
setup_clf = setup(data = train, target = 'target', fold_shuffle=True, use_gpu=True)

,Description,Value
0,session_id,7757
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2, 3: 3"
4,Original Data,"(2335, 33)"
5,Missing Values,False
6,Numeric Features,32
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


AttributeError: 'Simple_Imputer' object has no attribute 'fill_value_categorical'

In [6]:
top10 = compare_models(sort='Accuracy', n_select=10, fold = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8176,0.9579,0.8188,0.8199,0.8162,0.7569,0.7584,9.7920
lightgbm,Light Gradient Boosting Machine,0.8023,0.9479,0.8029,0.8048,0.8017,0.7364,0.7375,5.7320
et,Extra Trees Classifier,0.8011,0.9434,0.8025,0.8073,0.7981,0.7349,0.7386,0.5380
xgboost,Extreme Gradient Boosting,0.7901,0.9459,0.7904,0.7927,0.7893,0.7200,0.7213,1.7160
qda,Quadratic Discriminant Analysis,0.7797,0.9395,0.7794,0.7839,0.7771,0.7059,0.7085,0.0140
gbc,Gradient Boosting Classifier,0.7742,0.9333,0.7752,0.7767,0.7725,0.6989,0.7006,2.9700
rf,Random Forest Classifier,0.7546,0.9256,0.7554,0.7572,0.7509,0.6728,0.6758,0.7780
nb,Naive Bayes,0.6585,0.8881,0.6611,0.6718,0.6512,0.5448,0.5537,0.0080
ada,Ada Boost Classifier,0.6169,0.8249,0.6189,0.6280,0.6169,0.4894,0.4926,0.2060
dt,Decision Tree Classifier,0.5459,0.6971,0.5460,0.5498,0.5459,0.3944,0.3952,0.0440


In [8]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10[:5]]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8196,0.9468,0.8191,0.8265,0.8193,0.7591,0.7613
1,0.7645,0.9458,0.7632,0.7749,0.7643,0.6854,0.6887
2,0.7615,0.9313,0.7608,0.7690,0.7613,0.6815,0.6838
3,0.7523,0.9426,0.7523,0.7548,0.7521,0.6694,0.6702
4,0.8129,0.9420,0.8137,0.8131,0.8108,0.7505,0.7519
Mean,0.7821,0.9417,0.7818,0.7876,0.7816,0.7092,0.7112
SD,0.0282,0.0055,0.0285,0.0274,0.0278,0.0377,0.0377


In [9]:
blender_top10 = blend_models(estimator_list=tuned_top10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8720,0.9650,0.8709,0.8744,0.8717,0.8291,0.8301
1,0.8537,0.9583,0.8547,0.8552,0.8536,0.8048,0.8054
2,0.8354,0.9581,0.8355,0.8395,0.8367,0.7803,0.7808
3,0.8476,0.9637,0.8480,0.8488,0.8475,0.7966,0.7970
4,0.8405,0.9453,0.8410,0.8432,0.8398,0.7872,0.7885
5,0.8160,0.9495,0.8190,0.8138,0.8138,0.7547,0.7554
6,0.8098,0.9577,0.8119,0.8091,0.8080,0.7465,0.7475
7,0.8344,0.9663,0.8348,0.8338,0.8331,0.7791,0.7797
8,0.8221,0.9523,0.8225,0.8284,0.8172,0.7628,0.7674
9,0.8037,0.9608,0.8041,0.8045,0.8029,0.7382,0.7389


In [10]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 

In [11]:
prediction

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,...,sensor_25,sensor_26,sensor_27,sensor_28,sensor_29,sensor_30,sensor_31,sensor_32,Label,Score
0,0.067135,5.040383,-2.965405,-12.952880,-1.938465,-10.997802,1.853669,21.077862,7.963131,0.009773,...,1.889536,-3.926719,0.929041,4.999071,-7.963270,4.044370,-1.993315,10.855771,0,0.7087
1,6.890655,-12.010728,24.881907,38.943245,-4.757496,5.926453,7.923851,3.925545,4.012127,-6.921524,...,-16.940167,-11.169073,0.202975,6.846631,7.911875,-2.852819,56.028042,52.967175,0,0.7780
2,-4.809695,0.962863,-1.893647,-6.168084,-3.948282,0.977219,12.960620,-9.048958,5.969053,0.874370,...,-0.025761,2.981749,4.948704,-10.841968,-4.860024,-6.917495,2.842422,-3.081374,1,0.7331
3,0.018158,-3.993114,12.001667,22.960746,-2.121793,-6.002385,2.173744,1.045248,1.073869,-0.019322,...,0.903349,5.156367,40.827713,-1.160840,-1.215552,0.037914,-24.968873,-7.148603,3,0.7100
4,-7.841592,-2.129997,7.872448,-8.953428,28.080007,-5.019895,-10.922551,-15.045833,-56.981894,-8.831718,...,-6.741350,0.831967,11.905613,3.106004,46.899363,3.001353,-17.881074,-6.882390,2,0.8873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338,7.963652,7.973099,12.877346,11.015341,-0.963491,-3.926525,-30.875620,6.022538,-1.934085,-2.019713,...,-0.822110,3.043326,11.113028,11.160472,2.034703,-0.056564,-3.959809,22.986533,3,0.6940
9339,-7.974772,0.997146,8.011316,-10.007376,1.065419,-15.090632,1.916883,-2.981494,2.948488,12.012327,...,-4.971529,-9.957906,19.081232,-3.056299,-0.714567,-0.859710,7.950773,2.952029,1,0.5180
9340,4.035916,-10.138330,5.943174,8.080807,28.007780,4.981148,23.084630,-30.048268,9.877151,-2.048269,...,23.143486,4.913298,25.066640,4.044110,-1.948533,1.921691,13.952396,11.866490,0,0.4727
9341,8.920498,0.033165,20.058309,68.132886,4.037498,19.068331,-4.109146,-4.067740,-11.044263,1.134035,...,2.902505,-13.936057,3.058599,33.004942,2.932643,9.968502,-3.097104,7.966220,0,0.8438


In [12]:
prediction['Label']

0       0
1       0
2       1
3       3
4       2
       ..
9338    3
9339    1
9340    0
9341    0
9342    3
Name: Label, Length: 9343, dtype: int32

In [13]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

In [ ]:
# extratree => 0.87
# xgboost => 0.87

# 보팅 (extratree + LGBM + catboost)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [ ]:
LGBM = LGBMClassifier()
xtree = ExtraTreesClassifier()
ctb = ctb.CatBoostClassifier()

VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X, train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 정확도 계산 - test셋을 분리 후 정확도 보기

## 보팅으로 정확도 보기

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8373

## ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
xtree = ExtraTreesClassifier()
xtree.fit(X_train, y_train)

In [ ]:
pred = xtree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8030

## 보팅으로 정확도 보기2 (RF -> extratree)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
xtree = ExtraTreesClassifier()

VC = VotingClassifier(estimators=[('xt',xtree),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8415

## 랜덤포레스트로 정확도 보기

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.7709

## xgboost로 정확도 보기

In [ ]:
import xgboost as xgb 

In [ ]:
dtrain = xgb.DMatrix(data=X_train , label=y_train)
dtest = xgb.DMatrix(data=X_test , label=y_test)

In [ ]:
params = { 'max_depth':3,
           'eta': 0.1,   # 학습률
           'objective':'multi:softmax',
           'eval_metric':'merror',
           'early_stoppings':10
        }

# 학습 횟수
num_rounds = 100

In [ ]:
# 데이터를 학습용은 ‘train’ , 검증용은 ‘eval’로 표시
wlist = [(dtrain,'train'),(dtest,'eval') ]

In [ ]:
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)  #xgb.fit이 아닌 xgb.train
xgb_model

## Catboost

In [ ]:
import catboost as ctb

In [ ]:
model_CBC = ctb.CatBoostClassifier()

In [ ]:
model_CBC.fit(X_train, y_train)

In [ ]:
pred = model_CBC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

#예측 정확도: 0.8458

## 보팅으로 정확도 보기2 (xgboost => catboost)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [ ]:
LGBM = LGBMClassifier()
xtree = ExtraTreesClassifier()
ctb = ctb.CatBoostClassifier()

VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
pred

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8544

# 딥러닝 기반

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
train_y = np.asarray(train_y).astype('object')

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 문자열을 숫자로 변환
e = LabelEncoder()
e.fit(train_y)
Y = e.transform(train_y)
Y_encoded = tf.keras.utils.to_categorical(Y)

In [ ]:
Y_encoded

In [ ]:
model = Sequential()
model.add(Dense(40, input_dim=32, activation = 'relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(train_X, Y_encoded, epochs=100, batch_size=10)



In [ ]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(test)
# flatten : 데이터 배열이 몇 차원이든 모두 1차원으로 바꿔 읽기 쉽게 해 주는 함수

In [ ]:
Y_prediction

In [ ]:
value_list=[]
for i in range(len(Y_prediction)):
    maxindex = numpy.argmax(Y_prediction[i])
    value_list.append(maxindex)

In [ ]:
value_list

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, value_list)))


# 예측 정확도: 0.8544

In [ ]:
submission = pd.read_csv('submission.csv')

submission['target2'] = value_list

submission

submission.to_csv('submission.csv',index=False)